In [1]:
%pip install -q pyarrow==6.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 47.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.18.0 requires pyarrow>=10.0.1, but you have pyarrow 6.0.1 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 6.0.1 which is incompatible.


In [2]:
%pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
%pip install -q datasets

In [1]:
from tqdm import tqdm
import pandas as pd
import torch
from transformers import pipeline
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score
from datasets import load_dataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
%cd /content/drive/My Drive/tinkoff sirius
!ls

/content/drive/My Drive/tinkoff sirius
finetune-annazvereva  QA_model.ipynb		   train-00000-of-00001.parquet
QA_bert_sberquad      test-00000-of-00001.parquet  validation-00000-of-00001.parquet


In [10]:
train= pd.read_parquet('train-00000-of-00001.parquet')
val=pd.read_parquet('validation-00000-of-00001.parquet')
test= pd.read_parquet('test-00000-of-00001.parquet')

In [3]:
val=pd.read_parquet('validation-00000-of-00001.parquet')

Поскольку на это решение есть предобученные модели, возьмем их и дообучим на ресурсах Colab

In [11]:
train.head()

,id,title,context,question,answers
0,62310,SberChallenge,В протерозойских отложениях органические остат...,чем представлены органические остатки?,{'text': ['известковыми выделениями сине-зелён...
1,28101,SberChallenge,В протерозойских отложениях органические остат...,что найдено в кремнистых сланцах железорудной ...,"{'text': ['нитевидные водоросли, грибные нити'..."
2,48834,SberChallenge,В протерозойских отложениях органические остат...,что встречается в протерозойских отложениях?,"{'text': ['органические остатки'], 'answer_sta..."
3,83056,SberChallenge,В протерозойских отложениях органические остат...,что относится к числу древнейших растительных ...,{'text': ['скопления графито-углистого веществ...
4,5816,SberChallenge,В протерозойских отложениях органические остат...,как образовалось графито-углистое вещество?,{'text': ['в результате разложения Corycium en...


In [12]:
test.head()

,id,title,context,question,answers
0,18009,SberChallenge,Многоклеточный организм — внесистематическая к...,У каких организмов отсутствуют настоящие диффе...,"{'text': [''], 'answer_start': [-1]}"
1,54494,SberChallenge,Многоклеточный организм — внесистематическая к...,"Какие животные появились на Земле 2,1 миллиард...","{'text': [''], 'answer_start': [-1]}"
2,56411,SberChallenge,Многоклеточный организм — внесистематическая к...,Когда предположительно появились многоклеточны...,"{'text': [''], 'answer_start': [-1]}"
3,20672,SberChallenge,Многоклеточный организм — внесистематическая к...,Как называется внесистематическая категория жи...,"{'text': [''], 'answer_start': [-1]}"
4,45545,SberChallenge,Многоклеточный организм — внесистематическая к...,"У какой категории живых организмов, тело котор...","{'text': [''], 'answer_start': [-1]}"


## Mathnub/rubert-large-sberquad-1

In [4]:
pipe = pipeline("question-answering", model="Mathnub/rubert-large-sberquad-1", device=device)

In [14]:
pipe

In [15]:
pipe(question=input(), context=input())

abc
abc


{'score': 0.42443928122520447, 'start': 0, 'end': 3, 'answer': 'abc'}

In [5]:
def compute_exact_match(true_answer, pred_answer):
    return 1 if true_answer.strip() == pred_answer.strip() else 0

total_samples = len(val)
exact_match_count = 0
true_answers = []
pred_answers = []

for i in tqdm(range(total_samples), desc="Evaluating on val set"):
    row = val.iloc[i]
    question = row['question']
    context = row['context']
    true_answer = row['answers']['text'][0]

    output = pipe(question=question, context=context)
    pred_answer = output['answer']

    exact_match_count += compute_exact_match(true_answer, pred_answer)
    true_answers.append(true_answer)
    pred_answers.append(pred_answer)

exact_match_score = exact_match_count / total_samples
print(f"Exact Match score on test set: {exact_match_score:.4f}")

Evaluating on val set:   0%|          | 0/5036 [00:00<?, ?it/s]d:\miniconda\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Evaluating on val set: 100%|██████████| 5036/5036 [11:42<00:00,  7.17it/s]

Exact Match score on test set: 0.4938


In [6]:
total_samples

5036

In [7]:
exact_match_count

2487

In [8]:
f1_score_macro = f1_score(true_answers, pred_answers, average='macro')
print(f"F1-score (macro) on val set: {f1_score_macro:.4f}")

F1-score (macro) on val set: 0.3168


## annazvereva/sberquad_rubert_base

In [9]:
pipe = pipeline("question-answering", model="annazvereva/sberquad_rubert_base", device=device, tokenizer='DeepPavlov/rubert-base-cased')

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/709M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

d:\miniconda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
def compute_exact_match(true_answer, pred_answer):
    return 1 if true_answer.strip() == pred_answer.strip() else 0

total_samples = len(val)
exact_match_count = 0
true_answers = []
pred_answers = []

for i in tqdm(range(total_samples), desc="Evaluating on val set"):
    row = val.iloc[i]
    question = row['question']
    context = row['context']
    true_answer = row['answers']['text'][0]

    output = pipe(question=question, context=context)
    pred_answer = output['answer']

    exact_match_count += compute_exact_match(true_answer, pred_answer)
    true_answers.append(true_answer)
    pred_answers.append(pred_answer)

exact_match_score = exact_match_count / total_samples
print(f"Exact Match score on test set: {exact_match_score:.4f}")

Evaluating on val set: 100%|██████████| 5036/5036 [03:31<00:00, 23.86it/s]

Exact Match score on test set: 0.4954


In [11]:
f1_score_macro = f1_score(true_answers, pred_answers, average='macro')
print(f"F1-score (macro) on val set: {f1_score_macro:.4f}")

F1-score (macro) on val set: 0.3187


удивительно, но based справился с этой задачей лучше

## Fine Tuning

In [12]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModelForQuestionAnswering.from_pretrained("annazvereva/sberquad_rubert_base")

In [20]:
model

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

откат pyarrow до версии 6.0 позволил подгрузить dataset из transformers, что прекрасно

In [21]:
ds = load_dataset("kuznetsoffandrey/sberquad")

README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.43M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/4.93M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

In [22]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

### Подготовка текста

In [23]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride.
    # This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # truncate context, not the question
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context.
    # This will help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [24]:
tokenized_datasets = ds.map(prepare_train_features, batched=True, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/45328 [00:00<?, ? examples/s]

Map:   0%|          | 0/5036 [00:00<?, ? examples/s]

Map:   0%|          | 0/23936 [00:00<?, ? examples/s]

In [25]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

In [26]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 45544
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 5063
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 24022
    })
})

In [27]:
metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='macro')

### Аругементы

In [28]:
args = TrainingArguments(
    f"finetune-annazvereva/sberquad_rubert",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [29]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [30]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.922400,1.505900
2,0.602100,1.721278
3,0.387900,1.987013


TrainOutput(global_step=8541, training_loss=0.6369999163240951, metrics={'train_runtime': 10400.01, 'train_samples_per_second': 13.138, 'train_steps_per_second': 0.821, 'total_flos': 2.6776123849764864e+16, 'train_loss': 0.6369999163240951, 'epoch': 3.0})

In [32]:
best_ckpt_path = trainer.state.best_model_checkpoint

In [33]:
import os
# Загрузить лучшую модель
best_model = AutoModelForQuestionAnswering.from_pretrained(best_ckpt_path)
best_tokenizer = AutoTokenizer.from_pretrained(best_ckpt_path)

# Сохранить модель
save_dir = "QA_bert_sberquad"
os.makedirs(save_dir, exist_ok=True)
best_model.save_pretrained(save_dir)
best_tokenizer.save_pretrained(save_dir)

('QA_bert_sberquad/tokenizer_config.json',
 'QA_bert_sberquad/special_tokens_map.json',
 'QA_bert_sberquad/vocab.txt',
 'QA_bert_sberquad/added_tokens.json',
 'QA_bert_sberquad/tokenizer.json')

In [13]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Загрузить модель
model = AutoModelForQuestionAnswering.from_pretrained("QA_bert_sberquad")

# Загрузить токенизатор
tokenizer = AutoTokenizer.from_pretrained("QA_bert_sberquad")

In [22]:
def answer_question(question, context):
    # Закодировать вопрос и контекст
    inputs = tokenizer(question, context, return_tensors="pt")

    # Получить предсказание
    output = model(**inputs)

    # Извлечь начало и конец ответа
    start_index = torch.argmax(output.start_logits)
    end_index = torch.argmax(output.end_logits)

    # Декодировать ответ
    answer_tokens = inputs.input_ids[0, start_index:end_index+1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    return answer

question = "Что такое интерференция"
context = "Интерференция - сложение двух и более когерентных волн"

answer = answer_question(question, context)
print(f"Ответ: {answer}")

Ответ: сложение двух и более когерентных волн


In [24]:
from huggingface_hub import notebook_login

In [25]:
notebook_login()

In [27]:
model.push_to_hub("QA_bert_sberquad")

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Bogdan01m/QA_bert_sberquad/commit/4033b1675a6c779d79805c49bd4c6b8a8e520b1d', commit_message='Upload BertForQuestionAnswering', commit_description='', oid='4033b1675a6c779d79805c49bd4c6b8a8e520b1d', pr_url=None, pr_revision=None, pr_num=None)

In [28]:
tokenizer.push_to_hub("QA_bert_sberquad")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Bogdan01m/QA_bert_sberquad/commit/0161182dd489c25204b9f613fee10b8d71d69d42', commit_message='Upload tokenizer', commit_description='', oid='0161182dd489c25204b9f613fee10b8d71d69d42', pr_url=None, pr_revision=None, pr_num=None)

## Eval my model

In [5]:
pipe = pipeline("question-answering",model="Bogdan01m/QA_bert_sberquad", device=device)

In [6]:
def compute_exact_match(true_answer, pred_answer):
    return 1 if true_answer.strip() == pred_answer.strip() else 0

total_samples = len(val)
exact_match_count = 0
true_answers = []
pred_answers = []

for i in tqdm(range(total_samples), desc="Evaluating on val set"):
    row = val.iloc[i]
    question = row['question']
    context = row['context']
    true_answer = row['answers']['text'][0]

    output = pipe(question=question, context=context)
    pred_answer = output['answer']

    exact_match_count += compute_exact_match(true_answer, pred_answer)
    true_answers.append(true_answer)
    pred_answers.append(pred_answer)

exact_match_score = exact_match_count / total_samples
print(f"Exact Match score on test set: {exact_match_score:.4f}")

Evaluating on val set:   0%|          | 0/5036 [00:00<?, ?it/s]

d:\miniconda\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Evaluating on val set: 100%|██████████| 5036/5036 [03:38<00:00, 23.08it/s]

Exact Match score on test set: 0.4541


In [7]:
f1_score_macro = f1_score(true_answers, pred_answers, average='macro')
print(f"F1-score (macro) on val set: {f1_score_macro:.4f}")

F1-score (macro) on val set: 0.2851
